In [1]:
# Dependencies
import pymongo
import datetime
import pandas as pd

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.se_db

# Declare the collections
players = db.players_db
nonplayers = db.nonplayers_db

In [2]:
player_path = "datacsvs/CleanPlayerData.csv"
playerdata = pd.read_csv(player_path)

playerlist = playerdata['person'].unique()
player_ids = {p:(i+1) for i,p in enumerate(playerlist)}

gamelist = playerdata['Game'].unique()
game_ids = []

for i, g in enumerate(gamelist):
    if type(g) == str:
        curr_game = {
            'str':g,
            'id':i,
            'format':g[0:2],
            'num':g[2:]
        }
        game_ids.append(curr_game)
#     else:
#         print(g)


playerdata.head()

,person,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,gtype,GameNum,Game,Broken,Role
0,Cracknut,G,L,1.0,1.0,1.0,E,N,LG,1.0,LG1,N,Tineye
1,Edgedancer,G,L,2.0,2.0,1.0,L,N,LG,1.0,LG1,N,Cop
2,Mailliw73,G,L,4.0,4.0,1.0,L,N,LG,1.0,LG1,N,Regular
3,Aether,E,W,4.0,4.0,1.0,V,N,LG,1.0,LG1,N,Smoker
4,Claincy,E,W,-,-,0.0,S,N,LG,1.0,LG1,N,Vote Cancel


In [3]:
game_ids_df = pd.DataFrame(game_ids)
# game_ids_df.head()
# gotta add other stuff / combine this into game stats thing, also Broken, start & end date, title, GM/IM, 

In [4]:
realdata = playerdata[playerdata['Alignment'] != '0']
realdata.tail()

,person,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,gtype,GameNum,Game,Broken,Role
3691,Ashbringer,G,L,1.0,1.0,1.0,E,N,QF,48.0,QF48,N,-
3692,Shard of Reading,E,W,-,-,0.0,S,N,QF,48.0,QF48,N,-
3693,The Windrunner Supreme,G,L,4.0,4.0,1.0,E,N,QF,48.0,QF48,N,-
3694,Ventyl,G,L,-,-,0.0,S,N,QF,48.0,QF48,N,-
3695,Experience,G,L,1.0,1.0,1.0,L,N,QF,48.0,QF48,N,-


In [5]:
players.delete_many({})
# Verify results:
results = players.find()
for result in results:
    print(result)

In [24]:
realdata['Death'].unique()

array(['E', 'L', 'V', 'S', 'F', 'N', 'M', 'I', 'D', nan, 'L?', 'O', '?'],
      dtype=object)

In [25]:
stats_list = []


for p in realdata.iterrows():
#     print(p[1])
    row = p[1]


    try: 
        curr_id = int(game_ids_df[game_ids_df['str'] == row['Game']].index[0])
    except:
        print(row)
        
    pgstat = {
        'player_id': player_ids.get(row['person'], 0),
        'player_name': row['person'],
        'game_id': curr_id,
        'game_str': row['Game'],
        'alignment': row['Alignment'],
        'fOutcome': row['fOutcome'],       
    }
    if type(row['Death']) != str:
#         print(row['Death'])
        pgstat['death'] = '0'
    else:
        pgstat['death'] = row['Death']
    
    if row['HitNum'] != '-':
        pgstat['HitNum'] = row['HitNum']
        
        if row['Hit1'] != '-' :
            pgstat['Hit1'] = row['Hit1']
            pgstat['HitL'] = row['HitL']
            
    if row['Inactive'] == 'Y':
        pgstat['inactive'] = True
    else:
        pgstat['inactive'] = False
        
    if row['Role'] != '-':
        pgstat['role'] = row['Role']
    
    stats_list.append(pgstat)
    
    players.insert_one(pgstat)
    
    

In [26]:
# # Verify results:
# results = players.find()
# for result in results:
#     print(result)
from bson.json_util import dumps
cursor = players.find()
list_cur = list(cursor)
json_data = dumps(list_cur)
# Writing data to file data.json 
with open('datajsons/playerdata.json', 'w') as file: 
    file.write(json_data) 

In [27]:
# Verify results:
results = players.find()
for result in results:
    print(result)

{'_id': ObjectId('5ffd2d2295bedd8f264932e0'), 'player_id': 1, 'player_name': 'Cracknut', 'game_id': 0, 'game_str': 'LG1', 'alignment': 'G', 'fOutcome': 'L', 'death': 'E', 'HitNum': '1.0', 'Hit1': '1.0', 'HitL': '1.0', 'inactive': False, 'role': 'Tineye'}
{'_id': ObjectId('5ffd2d2295bedd8f264932e1'), 'player_id': 2, 'player_name': 'Edgedancer', 'game_id': 0, 'game_str': 'LG1', 'alignment': 'G', 'fOutcome': 'L', 'death': 'L', 'HitNum': '1.0', 'Hit1': '2.0', 'HitL': '2.0', 'inactive': False, 'role': 'Cop'}
{'_id': ObjectId('5ffd2d2295bedd8f264932e2'), 'player_id': 3, 'player_name': 'Mailliw73', 'game_id': 0, 'game_str': 'LG1', 'alignment': 'G', 'fOutcome': 'L', 'death': 'L', 'HitNum': '1.0', 'Hit1': '4.0', 'HitL': '4.0', 'inactive': False, 'role': 'Regular'}
{'_id': ObjectId('5ffd2d2295bedd8f264932e3'), 'player_id': 4, 'player_name': 'Aether', 'game_id': 0, 'game_str': 'LG1', 'alignment': 'E', 'fOutcome': 'W', 'death': 'V', 'HitNum': '1.0', 'Hit1': '4.0', 'HitL': '4.0', 'inactive': False,

{'_id': ObjectId('5ffd2d2495bedd8f26493932'), 'player_id': 192, 'player_name': 'Arinian', 'game_id': 69, 'game_str': 'LG29', 'alignment': 'N', 'fOutcome': 'W', 'death': 'S', 'HitNum': '0.0', 'inactive': False}
{'_id': ObjectId('5ffd2d2495bedd8f26493933'), 'player_id': 193, 'player_name': 'Manukos', 'game_id': 69, 'game_str': 'LG29', 'alignment': 'G', 'fOutcome': 'L', 'death': 'M', 'HitNum': '1.0', 'Hit1': '3.0', 'HitL': '4.0', 'inactive': False}
{'_id': ObjectId('5ffd2d2495bedd8f26493934'), 'player_id': 194, 'player_name': 'Droughtbringer', 'game_id': 69, 'game_str': 'LG29', 'alignment': 'B', 'fOutcome': 'W', 'death': 'S', 'HitNum': '0.0', 'inactive': False}
{'_id': ObjectId('5ffd2d2495bedd8f26493935'), 'player_id': 32, 'player_name': 'A Joe in the Bush', 'game_id': 70, 'game_str': 'MR19', 'alignment': 'E', 'fOutcome': 'W', 'death': 'S', 'HitNum': '1.0', 'Hit1': '2.0', 'HitL': '-', 'inactive': False}
{'_id': ObjectId('5ffd2d2495bedd8f26493936'), 'player_id': 45, 'player_name': 'Sart', 

{'_id': ObjectId('5ffd304d95bedd8f2649489b'), 'player_id': 63, 'player_name': 'Araris Valerian', 'game_id': 20, 'game_str': 'QF6', 'alignment': 'G', 'fOutcome': 'W', 'death': 'L', 'HitNum': '1.0', 'Hit1': '2.0', 'HitL': '2.0', 'inactive': False, 'role': 'Regular'}
{'_id': ObjectId('5ffd304d95bedd8f2649489c'), 'player_id': 69, 'player_name': 'Winter', 'game_id': 20, 'game_str': 'QF6', 'alignment': 'G', 'fOutcome': 'W', 'death': 'S', 'HitNum': '0.0', 'inactive': False, 'role': 'Matchmaker'}
{'_id': ObjectId('5ffd304d95bedd8f2649489d'), 'player_id': 74, 'player_name': 'Eolhondras', 'game_id': 20, 'game_str': 'QF6', 'alignment': 'G', 'fOutcome': 'W', 'death': 'S', 'HitNum': '0.0', 'inactive': False, 'role': 'Regular'}
{'_id': ObjectId('5ffd304d95bedd8f2649489e'), 'player_id': 81, 'player_name': 'Lord Pifferdoo', 'game_id': 20, 'game_str': 'QF6', 'alignment': 'E', 'fOutcome': 'L', 'death': 'L', 'HitNum': '1.0', 'Hit1': '4.0', 'HitL': '4.0', 'inactive': False, 'role': 'Regular'}
{'_id': Obje

{'_id': ObjectId('5ffd306b95bedd8f2649525e'), 'player_id': 219, 'player_name': 'Eternum', 'game_id': 99, 'game_str': 'AG4', 'alignment': 'G', 'fOutcome': 'L', 'death': 'F', 'HitNum': '1.0', 'Hit1': '2.0', 'HitL': '2.0', 'inactive': False}
{'_id': ObjectId('5ffd306b95bedd8f2649525f'), 'player_id': 234, 'player_name': 'Devotary of Spontaneity', 'game_id': 99, 'game_str': 'AG4', 'alignment': 'G', 'fOutcome': 'L', 'death': 'F', 'HitNum': '1.0', 'Hit1': '1.0', 'HitL': '1.0', 'inactive': False}
{'_id': ObjectId('5ffd306b95bedd8f26495260'), 'player_id': 238, 'player_name': 'MonsterMetroid', 'game_id': 99, 'game_str': 'AG4', 'alignment': 'G', 'fOutcome': 'L', 'death': 'E', 'HitNum': '1.0', 'Hit1': '6.0', 'HitL': '6.0', 'inactive': False}
{'_id': ObjectId('5ffd306b95bedd8f26495261'), 'player_id': 239, 'player_name': 'Fifth Scholar', 'game_id': 99, 'game_str': 'AG4', 'alignment': 'G', 'fOutcome': 'L', 'death': 'E', 'HitNum': '1.0', 'Hit1': '8.0', 'HitL': '8.0', 'inactive': False}
{'_id': ObjectI

{'_id': ObjectId('5ffd307395bedd8f26495cbc'), 'player_id': 3, 'player_name': 'Mailliw73', 'game_id': 56, 'game_str': 'MR15', 'alignment': 'E', 'fOutcome': 'L', 'death': 'L', 'inactive': False}
{'_id': ObjectId('5ffd307395bedd8f26495cbd'), 'player_id': 45, 'player_name': 'Sart', 'game_id': 56, 'game_str': 'MR15', 'alignment': 'E', 'fOutcome': 'L', 'death': 'M', 'inactive': False}
{'_id': ObjectId('5ffd307395bedd8f26495cbe'), 'player_id': 74, 'player_name': 'Eolhondras', 'game_id': 56, 'game_str': 'MR15', 'alignment': 'G', 'fOutcome': 'W', 'death': 'S', 'inactive': False}
{'_id': ObjectId('5ffd307395bedd8f26495cbf'), 'player_id': 77, 'player_name': 'Seonid', 'game_id': 56, 'game_str': 'MR15', 'alignment': 'E', 'fOutcome': 'L', 'death': 'L', 'inactive': False}
{'_id': ObjectId('5ffd307395bedd8f26495cc0'), 'player_id': 86, 'player_name': 'Kipper', 'game_id': 56, 'game_str': 'MR15', 'alignment': 'G', 'fOutcome': 'W', 'death': 'L', 'inactive': False}
{'_id': ObjectId('5ffd307395bedd8f26495cc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
nonplayer_path = "datacsvs/NonPlayerData.csv"
npdata = pd.read_csv(nonplayer_path)
npdata.tail()

,person,GM,Mod,Spec,gtype,GameNum,Game,Broken
1440,Straw,Y,N,N,LG,71.0,LG71,N
1441,Matrim's_Dice,Y,N,N,LG,71.0,LG71,N
1442,little wilson,N,Y,N,LG,71.0,LG71,N
1443,Elkanah,Y,N,N,QF,48.0,QF48,N
1444,Elbereth,N,Y,N,QF,48.0,QF48,N


In [9]:
nonplayers.delete_many({})
# Verify results:
results = nonplayers.find()
for result in results:
    print(result)

In [10]:
np_stats = []


for n in npdata.iterrows():
#     print(p[1])
    row2 = n[1]


    try: 
        curr_id2 = int(game_ids_df[game_ids_df['str'] == row2['Game']].index[0])
    except:
        print(row)
        
    npgstat = {
        'player_id': player_ids.get(row2['person'], 0),
        'player_name': row2['person'],
        'game_id': curr_id2,
        'game_str': row2['Game'],
                
    }
            
    if row2['Spec'] == 'Y':
        npgstat['GM'] = False
        npgstat['spec'] = True
        npgstat['IM'] = False
    elif row2['GM'] == 'Y':
        npgstat['GM'] = True
        npgstat['spec'] = False
        npgstat['IM'] = False
    else:
        npgstat['GM'] = False
        npgstat['spec'] = False
        npgstat['IM'] = True
        

    
    np_stats.append(npgstat)
    
    nonplayers.insert_one(npgstat)
    
    

In [11]:
# results = nonplayers.find()
# for result in results:
#     print(result)

cursor2 = nonplayers.find()
list_cur2 = list(cursor2)
json_data2 = dumps(list_cur2)
# Writing data to file data.json 
with open('datajsons/nonplayerdata.json', 'w') as file2: 
    file2.write(json_data2) 

In [13]:
len(stats_list[1])

13